In [1]:
# libraries
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import sqlite3

# modules




print(f"Current working directory: {os.getcwd()}")
# Change the current working directory to the parent directory
os.chdir('..')

# Print the new working directory to confirm the change
print("New working directory:", os.getcwd())
current_dir = Path.cwd()



Current working directory: /Users/mikey/code_projects/methane_project/notebooks
New working directory: /Users/mikey/code_projects/methane_project


In [2]:
#####################################################################################################################
## Modules
#####################################################################################################################

from src.dB_classes.manage_methane_db import MethaneDB
from src.dB_classes.image_wrangler import ImageWrangler
from src.dB_classes.fetch_data_from_api import FetchData
#from src.dB_classes.transform_data import TransformData
#from src.dB_classes.load_data import LoadDatas
from src.dB_classes.etl_pipeline import ETLPipeline

Current working directory: /Users/mikey/code_projects/methane_project


In [3]:
DATABASE = "methane_project_DB"
DB_FOLDER_PATH = current_dir / "data"

# set path to db file
PATH_TO_DB = DB_FOLDER_PATH / DATABASE

database = MethaneDB(PATH_TO_DB)    

#database.print_all_tables_and_values()
query='''SELECT * FROM photos'''
df = database.query_db(query)

In [6]:
import folium
print(folium.__version__)

0.16.0


In [5]:
df.iloc[0, 1]

b'<!doctype html><html lang="en-US" dir="ltr"><head><base href="https://accounts.google.com/v3/signin/"><link rel="preconnect" href="//www.gstatic.com"><meta name="referrer" content="origin"><style data-href="https://www.gstatic.com/_/mss/boq-identity/_/ss/k=boq-identity.AccountsSignInUi.FMSwV0kdA-M.L.W.O/am=BA0DBM0ZAVCAIwINABAAKAAAAAAAAAAABAZAQg/d=1/ed=1/rs=AOaEmlHsY1dLIiNgrMxB6koM0-lo0Td73g/m=identifierview,_b,_tp" nonce="bDO0gg7w3PwORfeoBwCpTQ">c-wiz{contain:style}c-wiz>c-data{display:none}c-wiz.rETSD{contain:none}c-wiz.Ubi8Z{contain:layout style}@-webkit-keyframes quantumWizBoxInkSpread{0%{-webkit-transform:translate(-50%,-50%) scale(0.2);-webkit-transform:translate(-50%,-50%) scale(0.2);transform:translate(-50%,-50%) scale(0.2)}to{-webkit-transform:translate(-50%,-50%) scale(2.2);-webkit-transform:translate(-50%,-50%) scale(2.2);transform:translate(-50%,-50%) scale(2.2)}}@keyframes quantumWizBoxInkSpread{0%{-webkit-transform:translate(-50%,-50%) scale(0.2);-webkit-transform:transl

In [ ]:

credentials_path = 'credentials.json'
GOOGLE_SHEET_ID = '1oJ2wAGYLkEd8VeKinrbiAmOwjlZpYqONL09P4LZ01po'
RANGE_NAME = 'Form Responses 1!A1:L'
DB_CONNECTION_STRING = 'sqlite:///mydatabase.db'
TABLE_NAME = 'responses'

gs2sql = GoogleSheetsToSQL(credentials_path, GOOGLE_SHEET_ID, RANGE_NAME, DB_CONNECTION_STRING)



In [ ]:
df  = gs2sql.read_sheet()

In [ ]:
df

In [ ]:
df[['latitude', 'longitude']] = df.loc[:, 'coordinates'].str.split(",", expand=True)
df.drop(columns='coordinates', inplace=True)

In [ ]:
df

In [ ]:

id_list = []
for idx, link in df['photo'].iterrows():
    if link:
        print(link)
        image = gs2sql.download_image_from_drive(link)
        image_id = gs2sql.insert_image_to_db(idx, image)
        id_list.append(image_id)
    else:
        id_list.append(None)
df['photo_id'] = pd.Series(id_list)
df.drop(columns='photo', inplace=True)
        

In [ ]:
def check_photos_table(db_path):
    try:
        # Connect to the SQLite database
        conn = sqlite3.connect(db_path)
        cur = conn.cursor()
        
        # Query the photos table
        cur.execute('SELECT hash FROM photos')
        rows = cur.fetchall()
        
        if rows:
            print("Contents of the photos table:")
            for row in rows:
                print(row)
        else:
            print("The photos table is empty.")
    
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    
    finally:
        # Close the connection
        if conn:
            conn.close()


check_photos_table("mydatabase.db")

In [ ]:
import requests
import folium
import json
import geopandas as gpd

# Define the API endpoint
url = "https://gis.portlandmaine.gov/maps/rest/services/Boundaries/Portland_Outline/MapServer/0/query"

# Define parameters for the API request
params = {
    'where': '1=1',
    'outFields': '*',
    'f': 'geojson'
}

# Send the request to the API
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    # Save the GeoJSON data to a file
    with open("portland_maine.geojson", "w") as f:
        json.dump(data, f, indent=2)
    print("GeoJSON data has been saved to 'portland_maine.geojson'")
else:
    print(f"Error fetching data: {response.status_code}")

# Create a Folium map centered around Portland, Maine
m = folium.Map(location=[43.6591, -70.2568], zoom_start=13)

# Add the GeoJSON layer to the map
folium.GeoJson(
    data,
    name="geojson"
).add_to(m)

# Add a layer control panel to the map
folium.LayerControl().add_to(m)

# Save the map as an HTML file
m.save("portland_maine_map.html")

# Display the map (in Jupyter Notebook or similar environment)
m

In [ ]:
 def get_base_map(self):

        # Define the API endpoint
        url = "https://gis.portlandmaine.gov/maps/rest/services/Boundaries/Portland_Outline/MapServer/0/query"

        # Define parameters for the API request
        params = {
            'where': '1=1',
            'outFields': '*',
            'f': 'geojson'
        }

        # Send the request to the API
        response = requests.get(url, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()
            # Save the GeoJSON data to a file
            with open("portland_maine.geojson", "w") as f:
                json.dump(data, f, indent=2)
            print("GeoJSON data has been saved to 'portland_maine.geojson'")
        else:
            print(f"Error fetching data: {response.status_code}")

        # Create a Folium map centered around Portland, Maine
        m = folium.Map(location=[43.6591, -70.2568], zoom_start=13)

        # Add the GeoJSON layer to the map
        folium.GeoJson(
                        data,
                        name="geojson"
                        ).add_to(self.map)

        # Add a layer control panel to the map
        folium.LayerControl().add_to(self.map)

        # Save the map as an HTML file
        self.map.save(self.map_name)
        print(f"Base map has been saved to {self.map_name}")